In [ ]:
!pip install numpngw

In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import numpngw

# Chemin du dossier contenant les images NIfTI
dossier_images_nii = '/content/drive/MyDrive/atlas-train-dataset-1.0.1/atlas-train-dataset-1.0.1/train/imagesTr'

# Chemin du dossier de sauvegarde des images PNG
dossier_images_png = '/content/drive/MyDrive/images'

# Vérifier si le dossier de sauvegarde existe, sinon le créer
if not os.path.exists(dossier_images_png):
    os.makedirs(dossier_images_png)

# Parcourir tous les fichiers dans le dossier
for fichier in os.listdir(dossier_images_nii):
    if fichier.endswith('.nii'):
        chemin_fichier_nii = os.path.join(dossier_images_nii, fichier)

        # Charger l'image NIfTI
        img = nib.load(chemin_fichier_nii)

        # Obtenir les données de l'image
        data = img.get_fdata()

        img = np.clip(data, -200, 30000) #Organic material >= 200HU (Hounsfield Units)

        for idx in range(img.shape[2]):
            cut = img[:,:,idx]
            cut = cut - np.min(cut) #Move image to positive numbers
            cut = np.stack((cut/(np.max(cut)+1e-8),)*3, axis=-1) #Normalized images to 3 channels
            cut = cut * np.iinfo(np.uint16).max #Images to 16 bits
            cut = cut.astype(np.uint16)[:,:,0]
            cut = np.expand_dims(cut, -1)
            # Sauvegarder l'image PNG avec le même nom de fichier
            chemin_sauvegarde_png = os.path.join(dossier_images_png, f"{fichier[:-7]}_{idx}.png")
            numpngw.write_png(chemin_sauvegarde_png, cut)

print("Conversion terminée pour tous les fichiers dans le dossier.")